### UserManager
    内置User模型引用模型管理器UserManager，其继承关系为：
       UserManager -> BaseUserManager -> models.Manager ...
    
    在UserManager中，定义了create_user以及create_superuser两种方法。从源代码中可以发现，其使用了username, email, is_staff, is_superuser，password等字段，is_superuser字段由PermissionsMixin定义, password字段由AbstractBaseUser定义。其余字段均在AbstractUser中定义。

In [ ]:
# 源代码
class UserManager(BaseUserManager):
    use_in_migrations = True

    def _create_user(self, username, email, password, **extra_fields):
        """
        Create and save a user with the given username, email, and password.
        """
        if not username:
            raise ValueError('The given username must be set')
        email = self.normalize_email(email)
        username = self.model.normalize_username(username)
        user = self.model(username=username, email=email, **extra_fields)
        user.set_password(password)
        user.save(using=self._db)
        return user

    def create_user(self, username, email=None, password=None, **extra_fields):
        extra_fields.setdefault('is_staff', False)
        extra_fields.setdefault('is_superuser', False)
        return self._create_user(username, email, password, **extra_fields)

    def create_superuser(self, username, email, password, **extra_fields):
        extra_fields.setdefault('is_staff', True)
        extra_fields.setdefault('is_superuser', True)

        if extra_fields.get('is_staff') is not True:
            raise ValueError('Superuser must have is_staff=True.')
        if extra_fields.get('is_superuser') is not True:
            raise ValueError('Superuser must have is_superuser=True.')

        return self._create_user(username, email, password, **extra_fields)

### BaseUserManager
    实现了三个方法。
    normalize_email()     将@后面的域名转换为小写
    make_random_password()  生成随机密码
    get_by_natural_key()   根据验证字段返回一个用户对象

### 重写UserManager
    倘若自定义了用户模型，则某些情况下需要对上述方法进行重写,可继承BaseUserManager。例如：
    以邮箱作为验证字段，而不是用户名，此时便需要对email的存在进行验证。
    或者自定义模型中没有is_superuser字段。(is_staff以及is_superuer字段与admin后台系统的使用密切相关，倘若缺乏，则无法使用后台管理系统。)